In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from utils.betting import load_current_line, load_unabated_current
pd.set_option("display.precision", 3)


In [2]:
corr3 = pd.read_csv('../reference_data/NFL/corr/corr3.csv',index_col=0)
play_corr = pd.read_csv('../reference_data/NFL//player_corr.csv',index_col=0)
corr3 = corr3.loc[corr3.count_all > 10]

In [3]:

today = dt.today()
year = today.year
month = today.month
day = today.day
today_str = f'{year}_{month}_{day}'
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 21)
pd.set_option('display.precision', 3)

pp_path = f'../Lines/pp/pp_{today_str}.parquet'
unabated_path = f'../Lines/unabated/unabated_{today_str}.parquet'
unabated = load_unabated_current(unabated_path, 'scrape_time', 'prop_id')
dk_path = f'../Lines/dk/dk_{today_str}.parquet'
dk = load_current_line(dk_path, 'scrape_time', 'prop_id')
pp_lines = load_current_line(pp_path, 'scrape_time', 'prop_id').drop_duplicates()
pp_lines = pp_lines.loc[
	pp_lines.league_id.isin([9])].reset_index(drop=True)

player_info = pd.read_csv('../reference_data/NFL/player_info.csv',index_col=0)
player_info = player_info.loc[player_info.year==2024]
pp = pp_lines.merge(player_info[['player','FantPos']],how='left')
pp_lines.scrape_time.max()

Timestamp('2024-10-18 17:10:00-0500', tz='US/Central')

In [4]:
dk.loc[dk.stat.str.contains('Rec')]['stat'].value_counts().head(25)

stat
Receiving Yards                           968
Rush + Rec TDs                            806
Receiving Yards Milestones                660
Longest Reception                         170
Receptions                                170
Rushing + Receiving Yards                 112
Rushing and Receiving Yards Milestones     97
Player Receiving Yards Total               84
Player Receiving Yards Spread              84
Player Receiving Yards ML                  70
Race to 20 Receiving Yards                 54
Race to 40 Receiving Yards                 54
Most Receiving Yards                       51
Team to Record Most Punts                  42
[Team2] to Record a Sack                   28
Most Rushing + Receiving Yards             26
Both Teams to Record a Defensive Sack      14
Name: count, dtype: int64

In [5]:
dk['stat'] = dk['stat'].replace(
{'Player Not to Score a Touchdown': 'Rush+Rec TDs',
 'Rushing + Receiving Yards': 'Rush+Rec Yds',
 'Rush + Rec TDs':'Rush+Rec TDs',
 'Score 2 or More': 'Rush+Rec TDs',
  'Anytime TD Scorer' : 'Rush+Rec TDs',
  'Anytime Touchdown Scorer' : 'Rush+Rec TDs',
  'Interceptions Thrown': 'INT',
  'Passing Yards': 'Pass Yards',
  'Passing Touchdowns': 'Pass TDs',
  'Rushing Yards':'Rush Yards',

   })
dk = dk.rename({'american_odds':'odds'},axis=1)
pp = pp.drop(['league_id','league_name','pp_player_id','prop_id','scrape_time','date','event_id'],axis=1,errors='ignore')
pp

,player,team,line,alt_line,stat,opp,event_time,FantPos
0,Khalil Shakir,BUF,20.5,goblin,Receiving Yards,TEN,2024-10-20 12:00:00-05:00,WR
1,Khalil Shakir,BUF,0.5,demon,Rush+Rec TDs,TEN,2024-10-20 12:00:00-05:00,WR
2,Khalil Shakir,BUF,59.5,demon,Receiving Yards,TEN,2024-10-20 12:00:00-05:00,WR
3,Khalil Shakir,BUF,32.5,standard,Receiving Yards,TEN,2024-10-20 12:00:00-05:00,WR
4,James Cook,BUF,0.5,demon,Rush+Rec TDs,TEN,2024-10-20 12:00:00-05:00,RB
...,...,...,...,...,...,...,...,...
1269,Kyren Williams,LA,79.5,goblin,Rush Yards,LV,2024-10-20 15:00:00-05:00,RB
1270,Kyren Williams,LA,29.5,demon,Receiving Yards,LV,2024-10-20 15:00:00-05:00,RB
1271,Kyren Williams,LA,14.5,standard,Receiving Yards,LV,2024-10-20 15:00:00-05:00,RB
1272,Kyren Williams,LA,90.5,standard,Rush Yards,LV,2024-10-20 15:00:00-05:00,RB


In [6]:
df = pp.merge(dk[['side','odds','prob','player','line','stat']],how='left', on=['player','line','stat'])
df.loc[df.alt_line=='standard'].sort_values(by='odds').dropna().tail(6)

,player,team,line,alt_line,stat,opp,event_time,FantPos,side,odds,prob
1496,Kareem Hunt,KC,14.5,standard,Receiving Yards,SF,2024-10-20 15:00:00-05:00,RB,under,−130,0.565
1324,Joe Burrow,CIN,10.5,standard,Rush Yards,CLE,2024-10-20 12:00:00-05:00,QB,under,−130,0.565
1511,Brock Purdy,SF,0.5,standard,INT,KC,2024-10-20 15:00:00-05:00,QB,under,−130,0.565
1018,Saquon Barkley,PHI,18.5,standard,Longest Rush,NYG,2024-10-20 12:00:00-05:00,RB,under,−130,0.565
1089,Daniel Jones,NYG,30.5,standard,Rush Yards,PHI,2024-10-20 12:00:00-05:00,QB,under,−135,0.574
374,Justin Jefferson,MIN,0.5,standard,Rush+Rec TDs,DET,2024-10-20 12:00:00-05:00,WR,over,−160,0.615


In [7]:
df['side'].value_counts()

side
over     366
under    261
Name: count, dtype: int64

In [8]:
overs = df.loc[(df.side.isin([np.NaN,'over'])) & (df.alt_line != 'goblin')]
overs

,player,team,line,alt_line,stat,opp,event_time,FantPos,side,odds,prob
2,Khalil Shakir,BUF,0.5,demon,Rush+Rec TDs,TEN,2024-10-20 12:00:00-05:00,WR,over,+215,0.317
3,Khalil Shakir,BUF,59.5,demon,Receiving Yards,TEN,2024-10-20 12:00:00-05:00,WR,NaN,NaN,NaN
4,Khalil Shakir,BUF,32.5,standard,Receiving Yards,TEN,2024-10-20 12:00:00-05:00,WR,NaN,NaN,NaN
5,James Cook,BUF,0.5,demon,Rush+Rec TDs,TEN,2024-10-20 12:00:00-05:00,RB,over,+100,0.500
6,James Cook,BUF,15.5,standard,Receiving Yards,TEN,2024-10-20 12:00:00-05:00,RB,over,−120,0.545
...,...,...,...,...,...,...,...,...,...,...,...
1529,Kyren Williams,LA,119.5,demon,Rush Yards,LV,2024-10-20 15:00:00-05:00,RB,NaN,NaN,NaN
1531,Kyren Williams,LA,29.5,demon,Receiving Yards,LV,2024-10-20 15:00:00-05:00,RB,NaN,NaN,NaN
1532,Kyren Williams,LA,14.5,standard,Receiving Yards,LV,2024-10-20 15:00:00-05:00,RB,NaN,NaN,NaN
1534,Kyren Williams,LA,90.5,standard,Rush Yards,LV,2024-10-20 15:00:00-05:00,RB,over,−115,0.535


In [9]:

pp_pairs = overs.merge(overs,on=['team','opp', 'event_time'], suffixes=['_1','_2'])
pp_pairs = pp_pairs.loc[pp_pairs.player_1!=pp_pairs.player_2]
p_corr = pp_pairs.merge(play_corr).drop(['event_time','opp'],axis=1)

In [10]:
#p_corr['both'] = p_corr['prob_1'] * p_corr['prob_2']
#p_corr['one'] = (1-p_corr['prob_1']) * p_corr['prob_2']


In [11]:
p_corr = p_corr.sort_values(by='w_corr').reset_index(drop=True)


In [12]:
pp_pairs

,player_1,team,line_1,alt_line_1,stat_1,opp,event_time,FantPos_1,side_1,odds_1,prob_1,player_2,line_2,alt_line_2,stat_2,FantPos_2,side_2,odds_2,prob_2
3,Khalil Shakir,BUF,0.5,demon,Rush+Rec TDs,TEN,2024-10-20 12:00:00-05:00,WR,over,+215,0.317,James Cook,0.5,demon,Rush+Rec TDs,RB,over,+100,0.500
4,Khalil Shakir,BUF,0.5,demon,Rush+Rec TDs,TEN,2024-10-20 12:00:00-05:00,WR,over,+215,0.317,James Cook,15.5,standard,Receiving Yards,RB,over,−120,0.545
5,Khalil Shakir,BUF,0.5,demon,Rush+Rec TDs,TEN,2024-10-20 12:00:00-05:00,WR,over,+215,0.317,James Cook,83.5,standard,Rush+Rec Yds,RB,over,−110,0.524
6,Khalil Shakir,BUF,0.5,demon,Rush+Rec TDs,TEN,2024-10-20 12:00:00-05:00,WR,over,+215,0.317,Amari Cooper,35.5,standard,Receiving Yards,WR,NaN,NaN,NaN
7,Khalil Shakir,BUF,0.5,demon,Rush+Rec TDs,TEN,2024-10-20 12:00:00-05:00,WR,over,+215,0.317,Keon Coleman,0.5,demon,Rush+Rec TDs,WR,over,+265,0.274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39931,Justin Tucker + Chase McLaughlin,BAL/TB,26.5,standard,Shortest FG Made Yds (Combo),TB/BAL,2024-10-21 19:00:00-05:00,NaN,NaN,NaN,NaN,Lamar Jackson + Baker Mayfield,484.5,standard,Pass Yards (Combo),NaN,NaN,NaN,NaN
39935,Justin Tucker + Chase McLaughlin,BAL/TB,47.5,standard,Longest FG Made Yds (Combo),TB/BAL,2024-10-21 19:00:00-05:00,NaN,NaN,NaN,NaN,Zay Flowers + Chris Godwin,138.5,standard,Receiving Yards (Combo),NaN,NaN,NaN,NaN
39936,Justin Tucker + Chase McLaughlin,BAL/TB,47.5,standard,Longest FG Made Yds (Combo),TB/BAL,2024-10-21 19:00:00-05:00,NaN,NaN,NaN,NaN,Lamar Jackson + Baker Mayfield,484.5,standard,Pass Yards (Combo),NaN,NaN,NaN,NaN
39940,Justin Tucker + Chase McLaughlin,BAL/TB,116.5,standard,Field Goal Yards (Combo),TB/BAL,2024-10-21 19:00:00-05:00,NaN,NaN,NaN,NaN,Zay Flowers + Chris Godwin,138.5,standard,Receiving Yards (Combo),NaN,NaN,NaN,NaN


In [27]:
noqb = p_corr.loc[p_corr.FantPos_1 != 'QB'].reset_index(drop=True)
noqb.loc[(noqb.gp > 8) & ~(noqb.stat_1.str.contains('TD')) & ~(noqb.stat_2.str.contains('TD'))].groupby(['player_1','player_2']).tail(5).sort_values(by='w_corr',ascending=True).reset_index(drop=True).iloc[0:20]

,player_1,team,line_1,alt_line_1,stat_1,FantPos_1,side_1,odds_1,prob_1,player_2,line_2,alt_line_2,stat_2,FantPos_2,side_2,odds_2,prob_2,corr,gp,pos_corr,w_corr
0,Justin Watson,KC,22.5,standard,Receiving Yards,WR,over,−115,0.535,JuJu Smith-Schuster,69.5,demon,Receiving Yards,WR,NaN,NaN,NaN,-0.435,20,-0.058,-0.247
1,JuJu Smith-Schuster,KC,40.5,standard,Receiving Yards,WR,NaN,NaN,NaN,Justin Watson,22.5,standard,Receiving Yards,WR,over,−115,0.535,-0.435,20,-0.058,-0.247
2,JuJu Smith-Schuster,KC,69.5,demon,Receiving Yards,WR,NaN,NaN,NaN,Justin Watson,22.5,standard,Receiving Yards,WR,over,−115,0.535,-0.435,20,-0.058,-0.247
3,Justin Watson,KC,22.5,standard,Receiving Yards,WR,over,−115,0.535,JuJu Smith-Schuster,40.5,standard,Receiving Yards,WR,NaN,NaN,NaN,-0.435,20,-0.058,-0.247
4,James Cook,BUF,83.5,standard,Rush+Rec Yds,RB,over,−110,0.524,Dalton Kincaid,59.5,demon,Receiving Yards,TE,NaN,NaN,NaN,-0.439,23,-0.021,-0.244
5,James Cook,BUF,83.5,standard,Rush+Rec Yds,RB,over,−110,0.524,Dalton Kincaid,39.5,standard,Receiving Yards,TE,NaN,NaN,NaN,-0.439,23,-0.021,-0.244
6,Justin Watson,KC,14.5,standard,Longest Reception,WR,over,−120,0.545,JuJu Smith-Schuster,40.5,standard,Receiving Yards,WR,NaN,NaN,NaN,-0.442,20,-0.047,-0.244
7,Justin Watson,KC,14.5,standard,Longest Reception,WR,over,−120,0.545,JuJu Smith-Schuster,69.5,demon,Receiving Yards,WR,NaN,NaN,NaN,-0.442,20,-0.047,-0.244
8,JuJu Smith-Schuster,KC,40.5,standard,Receiving Yards,WR,NaN,NaN,NaN,Justin Watson,14.5,standard,Longest Reception,WR,over,−120,0.545,-0.442,20,-0.047,-0.244
9,JuJu Smith-Schuster,KC,69.5,demon,Receiving Yards,WR,NaN,NaN,NaN,Justin Watson,14.5,standard,Longest Reception,WR,over,−120,0.545,-0.442,20,-0.047,-0.244


In [32]:
a = pd.concat(
    (p_corr.sort_values(by='w_corr',ascending=True).groupby(['player_1','player_2','stat_1','stat_2']).head(1),
     p_corr.sort_values(by='w_corr',ascending=True).groupby(['player_1','player_2','stat_1','stat_2']).tail(1)
    )
)
a = a[~(a.stat_1.str.contains('TD')) & ~(a.stat_2.str.contains('TD'))]
a.sort_values(by='pos_corr',ascending=False).reset_index(drop=True).groupby(['player_1','player_2']).head(2).head(25)

,player_1,team,line_1,alt_line_1,stat_1,FantPos_1,side_1,odds_1,prob_1,player_2,line_2,alt_line_2,stat_2,FantPos_2,side_2,odds_2,prob_2,corr,gp,pos_corr,w_corr
0,Jalen Hurts,PHI,224.5,demon,Pass Yards,QB,NaN,NaN,NaN,A.J. Brown,73.5,standard,Receiving Yards,WR,NaN,NaN,NaN,0.729,37,0.227,0.553
1,Lamar Jackson,BAL,228.5,standard,Pass Yards,QB,NaN,NaN,NaN,Rashod Bateman,59.5,demon,Receiving Yards,WR,NaN,NaN,NaN,0.029,36,0.227,0.100
2,Patrick Mahomes,KC,274.5,demon,Pass Yards,QB,NaN,NaN,NaN,Justin Watson,22.5,standard,Receiving Yards,WR,over,−115,0.535,-0.086,41,0.227,0.017
3,C.J. Stroud,HOU,258.5,standard,Pass Yards,QB,NaN,NaN,NaN,Stefon Diggs,99.5,demon,Receiving Yards,WR,NaN,NaN,NaN,0.032,6,0.227,0.182
4,Jared Goff,DET,250.5,standard,Pass Yards,QB,NaN,NaN,NaN,Jameson Williams,69.5,demon,Receiving Yards,WR,NaN,NaN,NaN,0.161,26,0.227,0.189
5,Brock Purdy,SF,274.5,demon,Pass Yards,QB,NaN,NaN,NaN,Jauan Jennings,31.5,standard,Receiving Yards,WR,NaN,NaN,NaN,0.167,32,0.227,0.190
6,Trevor Lawrence,JAC,249.5,demon,Pass Yards,QB,NaN,NaN,NaN,Christian Kirk,43.5,standard,Receiving Yards,WR,NaN,NaN,NaN,0.175,37,0.227,0.193
7,Aidan O'Connell,LV,211.5,standard,Pass Yards,QB,NaN,NaN,NaN,Tre Tucker,35.5,standard,Receiving Yards,WR,over,−120,0.545,0.143,13,0.227,0.194
8,Josh Allen,BUF,224.5,demon,Pass Yards,QB,NaN,NaN,NaN,Keon Coleman,25.5,standard,Receiving Yards,WR,over,−115,0.535,0.087,6,0.227,0.195
9,Patrick Mahomes,KC,240.5,standard,Pass Yards,QB,over,−115,0.535,Justin Watson,22.5,standard,Receiving Yards,WR,over,−115,0.535,-0.086,41,0.227,0.017


In [15]:
p_corr.loc[p_corr.team=='DEN'].sort_values(by='w_corr',ascending=False).reset_index(drop=True).head(20)

,player_1,team,line_1,alt_line_1,stat_1,FantPos_1,side_1,odds_1,prob_1,player_2,line_2,alt_line_2,stat_2,FantPos_2,side_2,odds_2,prob_2,corr,gp,pos_corr,w_corr


In [16]:
a = corr3.merge(df[['player','stat','line','prob','alt_line','side', 'FantPos','team']],
            left_on=['alt_line_1','stat_1','FantPos_1','o_u_1'],
            right_on=['alt_line','stat','FantPos','side'],how='left'
            )
a.loc[a.stat_1.str.contains('Rec TDs')]

,alt_line_1,alt_line_2,alt_line_3,stat_1,stat_2,stat_3,FantPos_1,FantPos_2,FantPos_3,o_u_1,...,relation,equiv,player,stat,line,prob,alt_line,side,FantPos,team
20,standard,standard,standard,Rush+Rec TDs,Longest Reception,Pass TDs,RB,WR,QB,under,...,same,0.774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,standard,standard,standard,Rush+Rec TDs,Rush Attempts,Receiving Yards,WR,RB,TE,over,...,same,0.741,Justin Jefferson,Rush+Rec TDs,0.5,0.615,standard,over,WR,MIN
231,standard,standard,standard,Rush+Rec TDs,Pass Yards,Longest Reception,RB,QB,WR,under,...,same,0.689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273,standard,standard,demon,Rush+Rec TDs,Longest Reception,FG,RB,WR,K,under,...,same,0.686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339,standard,standard,standard,Rush+Rec TDs,Receptions,Fantasy Score,RB,WR,TE,over,...,same,0.682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195050,demon,standard,standard,Rush+Rec TDs,Rush Attempts,Receptions,WR,RB,WR,over,...,same,0.220,Chris Godwin,Rush+Rec TDs,0.5,0.385,demon,over,WR,TB
195051,demon,standard,standard,Rush+Rec TDs,Rush Attempts,Receptions,WR,RB,WR,over,...,same,0.220,Rashod Bateman,Rush+Rec TDs,0.5,0.260,demon,over,WR,BAL
195052,demon,standard,standard,Rush+Rec TDs,Rush Attempts,Receptions,WR,RB,WR,over,...,same,0.220,Zay Flowers,Rush+Rec TDs,0.5,0.377,demon,over,WR,BAL
195053,demon,standard,standard,Rush+Rec TDs,Rush Attempts,Receptions,WR,RB,WR,over,...,same,0.220,Allen Lazard,Rush+Rec TDs,0.5,0.244,demon,over,WR,NYJ


In [17]:
bets = corr3.merge(df[['player','stat','line','prob','alt_line','side', 'FantPos','team']],
            left_on=['alt_line_1','stat_1','FantPos_1','o_u_1'],
            right_on=['alt_line','stat','FantPos','side']
            ).drop(['stat','alt_line','side','FantPos'],axis=1).merge(
                df[['player','stat','line','prob','alt_line','side', 'FantPos','team']],
            left_on=['alt_line_2','stat_2','FantPos_2','o_u_2','team'],
            right_on=['alt_line','stat','FantPos','side','team'],
            suffixes=['_1','_2']
            ).drop(['stat','alt_line','side','FantPos'],axis=1).merge(
                df[['player','stat','line','prob','alt_line','side', 'FantPos','team']],
            left_on=['alt_line_3','stat_3','FantPos_3','o_u_3','team'],
            right_on=['alt_line','stat','FantPos','side','team'],
            suffixes=['','_3']
            ).drop(['stat','alt_line','side','FantPos'],axis=1)
bets = bets.loc[bets.player_1 != bets.player_2]
bets = bets.loc[bets.player != bets.player_2]
bets = bets.loc[bets.player_1 != bets.player]

In [18]:
pd.set_option('display.max_columns', 30)
bets['prob_1'] = bets['prob_1'].fillna(0.25)
bets['prob_2'] = bets['prob_2'].fillna(0.25)
bets['prob'] =bets['prob'].fillna(0.25)
bets['prob_all'] = (bets['prob_1'] * bets['prob_2'] * bets['prob'])
bets['exp'] = (bets['proportion'] + bets['prob_all']) * 0.5
bets = bets.drop(['FantPos_1','FantPos_2','FantPos_3'],axis=1)

In [19]:
bets.loc[bets.team=='NO'].sort_values(by='proportion',ascending=False).head(15)

,alt_line_1,alt_line_2,alt_line_3,stat_1,stat_2,stat_3,o_u_1,o_u_2,o_u_3,proportion,count_all,relation,equiv,player_1,line_1,prob_1,team,player_2,line_2,prob_2,player,line,prob,prob_all,exp


In [20]:
bets.loc[bets.count_all > 30].sort_values(by='exp',ascending=False).groupby(['stat_1','stat_2','stat_3']).head(4).groupby(['team']).head(3)

,alt_line_1,alt_line_2,alt_line_3,stat_1,stat_2,stat_3,o_u_1,o_u_2,o_u_3,proportion,count_all,relation,equiv,player_1,line_1,prob_1,team,player_2,line_2,prob_2,player,line,prob,prob_all,exp
24627,standard,standard,standard,Pass Yards,Receptions,Longest Reception,under,under,under,0.284,33,same,0.658,Kirk Cousins,267.5,0.535,ATL,Kyle Pitts,3.5,0.565,Drake London,22.5,0.535,0.162,0.223
22624,standard,standard,standard,Longest Rush,Longest Rush,Longest Reception,under,over,over,0.270,34,same,0.646,Tyler Allgeier,13.5,0.556,ATL,Bijan Robinson,16.5,0.545,Drake London,22.5,0.535,0.162,0.216
14621,standard,standard,standard,Longest Rush,Longest Rush,Longest Reception,under,over,over,0.270,34,same,0.646,Zack Moss,11.5,0.565,CIN,Chase Brown,16.5,0.535,Andrei Iosivas,13.5,0.535,0.162,0.216
25145,standard,standard,standard,Longest Reception,Receptions,Pass Yards,under,under,under,0.259,35,same,0.638,Kyle Pitts,18.5,0.535,ATL,Drake London,6.5,0.556,Kirk Cousins,267.5,0.535,0.159,0.209
14622,standard,standard,standard,Longest Rush,Longest Rush,Longest Reception,under,over,over,0.270,34,same,0.646,Chase Brown,16.5,0.535,CIN,Zack Moss,11.5,0.500,Andrei Iosivas,13.5,0.535,0.143,0.206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15752,standard,standard,demon,Rush+Rec Yds,Receiving Yards,INT,under,under,over,0.095,40,same,0.456,James Cook,83.5,0.545,BUF,Keon Coleman,25.5,0.535,Josh Allen,0.5,0.488,0.142,0.118
26789,demon,standard,standard,Rush+Rec TDs,Longest Reception,Longest Reception,over,under,under,0.072,88,same,0.415,Austin Ekeler,0.5,0.465,WAS,Noah Brown,17.5,0.524,Terry McLaurin,24.5,0.535,0.130,0.101
26518,demon,standard,demon,Rush+Rec TDs,Rush Yards,Pass TDs,over,over,over,0.098,55,same,0.462,Ladd McConkey,0.5,0.339,LAC,J.K. Dobbins,76.5,0.565,Justin Herbert,1.5,0.377,0.072,0.085
26516,demon,standard,demon,Rush+Rec TDs,Rush Yards,Pass TDs,over,under,over,0.089,50,same,0.447,Ladd McConkey,0.5,0.339,LAC,J.K. Dobbins,76.5,0.500,Justin Herbert,1.5,0.377,0.064,0.077


In [21]:
bets2 = corr3.merge(pp[['player','stat','line','alt_line', 'FantPos','team']],
            left_on=['alt_line_1','stat_1','FantPos_1'],
            right_on=['alt_line','stat','FantPos']
            ).drop(['stat','alt_line','FantPos'],axis=1).merge(
                pp[['player','stat','line','alt_line', 'FantPos','team']],
            left_on=['alt_line_2','stat_2','FantPos_2','team'],
            right_on=['alt_line','stat','FantPos','team'],
            suffixes=['_1','_2']
            ).drop(['stat','alt_line','FantPos'],axis=1).merge(
                pp[['player','stat','line','alt_line', 'FantPos','team']],
            left_on=['alt_line_3','stat_3','FantPos_3','team'],
            right_on=['alt_line','stat','FantPos','team'],
            suffixes=['','_3']
            ).drop(['stat','alt_line','FantPos'],axis=1)

In [22]:
bets2.loc[(bets2.count_all > 40) & (
    (bets2.alt_line_1 == 'demon') |
    (bets2.alt_line_2 == 'demon') |
    (bets2.alt_line_3 == 'demon')
)].sort_values(by='equiv',ascending=False).groupby(['stat_1','stat_2','stat_3']).head(2).head(20)

,alt_line_1,alt_line_2,alt_line_3,stat_1,stat_2,stat_3,FantPos_1,FantPos_2,FantPos_3,o_u_1,o_u_2,o_u_3,proportion,count_all,relation,equiv,player_1,line_1,team,player_2,line_2,player,line
58067,demon,standard,standard,Pass Yards,Longest Reception,Fantasy Score,QB,WR,WR,over,over,over,0.202,49,same,0.587,Kirk Cousins,274.5,ATL,Darnell Mooney,21.5,Darnell Mooney,12.0
312993,demon,standard,standard,Pass Yards,Longest Reception,Fantasy Score,QB,WR,WR,over,over,over,0.202,49,same,0.587,Daniel Jones,224.5,NYG,Wan'Dale Robinson,17.5,Malik Nabers,16.5
498701,standard,standard,demon,Longest Reception,Longest Reception,Pass Yards,RB,WR,QB,over,over,over,0.165,42,same,0.549,Chase Brown,10.5,CIN,Tee Higgins,22.5,Joe Burrow,274.5
541188,standard,standard,demon,Longest Reception,Longest Reception,Pass Yards,RB,WR,QB,over,over,over,0.165,42,same,0.549,Saquon Barkley,9.5,PHI,A.J. Brown,24.5,Jalen Hurts,224.5
361727,standard,demon,standard,Rec Targets,Pass Yards,Longest Reception,TE,QB,WR,over,over,over,0.165,41,same,0.548,Isaiah Likely,3.0,BAL,Lamar Jackson,274.5,Zay Flowers,22.5
361719,standard,demon,standard,Rec Targets,Pass Yards,Longest Reception,TE,QB,WR,over,over,over,0.165,41,same,0.548,Mark Andrews,3.5,BAL,Lamar Jackson,249.5,Rashod Bateman,17.5
427799,standard,demon,standard,Rec Targets,Rush+Rec TDs,Pass+Rush Yds,TE,WR,QB,under,over,over,0.163,41,same,0.546,Cade Otton,5.5,TB,Mike Evans,0.5,Baker Mayfield,275.5
427798,standard,demon,standard,Rec Targets,Rush+Rec TDs,Pass+Rush Yds,TE,WR,QB,under,over,over,0.163,41,same,0.546,Cade Otton,5.5,TB,Mike Evans,0.5,Baker Mayfield,275.5
68184,standard,demon,standard,Receptions,Pass TDs,Longest Reception,RB,QB,WR,under,over,over,0.158,58,same,0.540,Bijan Robinson,3.5,ATL,Kirk Cousins,2.5,Drake London,22.5
485095,standard,demon,standard,Receptions,Pass TDs,Longest Reception,RB,QB,WR,under,over,over,0.158,58,same,0.540,Tony Pollard,3.5,TEN,Will Levis,1.5,DeAndre Hopkins,18.5


In [23]:
df.loc[df.alt_line=='standard'].sort_values(by='american_odds').dropna()

KeyError: 'american_odds'

In [123]:
dk.loc[(dk.stat=='Receiving Yards') & (dk['player'].str.contains('Aiyu'))]

,side,decimal_odds,outcomeType,american_odds,eventId,name,stat,line,player,home,away,event_time,alt_line,prob,scrape_time,date,prop_id,event_id,league_name


In [124]:
deeb = dk.loc[(dk['player'].str.contains('Deebo', na=False)) & (dk['stat'] == 'Receiving Yards')][
    ['player','side','american_odds','line','prob']].sort_values(by=['side','prob'],ascending=False)

In [125]:
deeb2 = deeb.loc[deeb.side=='over'].merge(deeb.loc[deeb.side=='under'],on=['player','line'], suffixes=['_o','_u'])

In [126]:
deeb2['fair'] = deeb2['prob_o'] / (deeb2['prob_u'] + deeb2['prob_o'])
deeb2

,side_o,american_odds_o,prob_o,player,side_u,american_odds_u,line,prob_u,fair
